In [6]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [7]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-10.351141,18.468288,-1.587198,5.859317,6.370951,-15.148269,4.167015,1.770694,16.307508,-18.527201,...,40.539562,-5.868604,-0.525147,29.252869,-18.695196,-24.255198,24.902847,1,12150,critical
1,-0.611337,1.667067,-0.084088,0.345362,0.262180,-0.930403,0.067351,0.382352,1.096432,-1.287243,...,2.431135,-0.560943,0.054394,2.061535,-1.769360,-1.742720,1.817466,1,10168,critical
2,-25.163929,32.384472,-5.075072,14.601456,21.233459,-35.906075,11.761786,4.701000,33.779083,-38.444489,...,89.350410,-8.292704,-4.148782,66.433205,-30.335083,-53.939602,48.110039,1,12783,critical
3,-13.665822,26.740517,-6.125227,11.503922,20.571180,-28.369169,9.178022,3.348720,27.385149,-33.969685,...,81.852448,-11.761877,0.143565,51.751614,-27.834795,-45.927044,45.104542,1,12754,critical
4,-15.209062,20.849018,-4.461039,6.608896,12.249491,-21.742653,7.111282,2.520332,18.878557,-22.469709,...,51.909943,-5.237012,-1.983984,37.225739,-20.388968,-32.374943,29.900106,1,4277,critical


In [8]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [9]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8296
F1_macro: 0.5713
Accuracy: 0.8296
Precision_macro: 0.5921
Recall_macro: 0.5636

Bagging Metrics:
F1_micro: 0.8442
F1_macro: 0.5503
Accuracy: 0.8442
Precision_macro: 0.6007
Recall_macro: 0.5442

XGBoost Metrics:
F1_micro: 0.8300
F1_macro: 0.5725
Accuracy: 0.8300
Precision_macro: 0.5926
Recall_macro: 0.5654


In [10]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")


Final Test Results:

Random Forest Metrics:
F1_micro: 0.8196
F1_macro: 0.5621
Precision_macro: 0.5745
Recall_macro: 0.5562
Accuracy: 0.8196
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       786
           1       0.27      0.20      0.23       123

    accuracy                           0.82       909
   macro avg       0.57      0.56      0.56       909
weighted avg       0.80      0.82      0.81       909


Bagging Metrics:
F1_micro: 0.8438
F1_macro: 0.5665
Precision_macro: 0.6089
Recall_macro: 0.5565
Accuracy: 0.8438
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       786
           1       0.34      0.16      0.22       123

    accuracy                           0.84       909
   macro avg       0.61      0.56      0.57       909
weighted avg       0.81      0.84      0.82       909


XGBoost Metrics:
F1_micro: 0.8229
F1_macro: 

In [13]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/codebert/result_codebert_base_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")


Results saved to /Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/codebert/result_codebert_base_sum.pkl
